Developed by Megan Renz, Rebeckah Fussell, and Natasha Holmes for Cornell Physics Labs.

### Fitting

In [ ]:
import sys

%matplotlib inline
#%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import csv
from scipy.optimize import minimize, rosen, rosen_der, curve_fit
from matplotlib.widgets import TextBox
import pandas as pd

# def chiSquared2(args,x, y, dy, f ):
#     '''Function Chi-Squared.
#     x, y and dy are numpy arrays, referring to x, y and the uncertainty in y respectively.
#     f is the function we are fitting.
#     args are the arguments of the function we have fit.
#     '''
#     return 1/(len(x)*np.sum((f(x, args)-y)**2/dy**2))

def chiSquared(x, y, dy, f, args):
    '''Function Chi-Squared.
    x, y and dy are numpy arrays, referring to x, y and the uncertainty in y respectively.
    f is the function we are fitting.
    args are the arguments of the function we have fit.
    '''
    return 1/(len(x))*np.sum((f(x, args)-y)**2/dy**2)

def linear(x,args):
    '''
    A special case of Poly.
    '''
    return x*args[0]+args[1]

def linearFit(x, m,b):
    return m*x+b

def poly(x, args):
    '''
    returns the value of the polynomial sum (x**i*args[i])
    '''
    total=x**0*args[0]
    for i in range(1,len(args)):
        total+=x**i*args[i]
    return total

def polyFit(x, A, B, C):
  return A*(x**2)+B*x+C

def autoFit(x=[], y=[], dy=[], title="Use title= in your call", xaxis="Use xaxis= in your call", yaxis="Use yaxis= in your call"):
    x0=[0,0]
    x=np.array(x)
    y=np.array(y)
    dy=np.array(dy)

    args,cov=curve_fit(linearFit, x, y, x0, dy)

    m=args[0]
    b=args[1]
    dm = np.sqrt(cov[0][0])
    db = np.sqrt(cov[1][1])

    fig, ax=plt.subplots(1,2, figsize=(12,6), dpi=75)
    line,=ax[0].plot(x,linear(x, [m,b]))
    data=ax[0].errorbar(x,y, dy, fmt='.k')
    ax[0].set_title(title)
    ax[0].set_xlabel(xaxis)
    ax[0].set_ylabel(yaxis)
    ax[1].set_title("Residuals")
    ax[1].set_xlabel(xaxis)
    ax[1].set_ylabel("point -- line")
    residuals=y-linear(x, [m,b])
    res=ax[1].errorbar(x,residuals, dy, fmt='.k')
    ax[1].grid(True, which='both')
    plt.show()

    print("chi-squared value =  {}".format(chiSquared(x,y, dy, linear, [m, b]).round(3)))
    print("Best fit line :  ")
    print("y=", round(args[0],4),"x", "+", round(args[1],4))
    print("m={}+\-{}".format(m.round(3),dm.round(3),))
    print("b={}+\-{}".format(b.round(3),db.round(3),))

def manualFit(x=[], y=[], dy=[], m=[], b=[], title="Use title= in your call", xaxis="Use xaxis= in your call", yaxis="Use yaxis= in your call"):
    x0=[0,0]
    x=np.array(x)
    y=np.array(y)
    dy=np.array(dy)

    fig, ax=plt.subplots(1,2, figsize=(12,6), dpi=75)
    line,=ax[0].plot(x,linear(x, [m,b]))
    data=ax[0].errorbar(x,y, dy, fmt='.k')
    ax[0].set_title(title)
    ax[0].set_xlabel(xaxis)
    ax[0].set_ylabel(yaxis)
    ax[1].set_title("Residuals")
    ax[1].set_xlabel(xaxis)
    ax[1].set_ylabel("point -- line")
    residuals=y-linear(x, [m,b])
    res=ax[1].errorbar(x,residuals, dy, fmt='.k')
    ax[1].grid(True, which='both')
    plt.show()

    print("chi-squared value =  {}".format(chiSquared(x,y, dy, linear, [m, b]).round(3)))
    print("Best fit line :  ")
    print("y=", m,"x", "+", b)


def autoFit2(x=[], y=[], dy=[], title="Use title= in your call", xaxis="Use xaxis= in your call", yaxis="Use yaxis= in your call"):
    x0=[0,0]
    x=np.array(x)
    y=np.array(y)
    dy=np.array(dy)

    args,cov=curve_fit(linearFit, x**2, y, x0, dy)

    m=args[0]
    b=args[1]
    dm = np.sqrt(cov[0][0])
    db = np.sqrt(cov[1][1])

    fig, ax=plt.subplots(1,2, figsize=(12,6), dpi=75)
    line,=ax[0].plot(x,linear(x**2, [m,b]))
    data=ax[0].errorbar(x,y, dy, fmt='.k')
    ax[0].set_title(title)
    ax[0].set_xlabel(xaxis)
    ax[0].set_ylabel(yaxis)
    ax[1].set_title("Residuals")
    ax[1].set_xlabel(xaxis)
    ax[1].set_ylabel("point -- line")
    residuals=y-linear(x**2, [m,b])
    res=ax[1].errorbar(x,residuals, dy, fmt='.k')
    ax[1].grid(True, which='both')
    plt.show()

    print("chi-squared value =  {}".format(chiSquared(x**2,y, dy, linear, [m, b]).round(3)))
    print("Best fit line :  ")
    print("y=", round(args[0],4),"x^2", "+", round(args[1],4))
    print("m={}+\-{}".format(m.round(3),dm.round(3),))
    print("b={}+\-{}".format(b.round(3),db.round(3),))


## Uncertainty and $t^\prime$

In [ ]:
def standard_deviation(data):
    '''
    Takes in a one-dimensional numpy array argument, and returns the standard deviation of the dataset.
    '''
    N = len(data)
    return np.sqrt(np.sum((data - np.mean(data))**2)/(N-1))

def standard_unc_of_mean(data):
    '''
    Takes in a one-dimensional numpy array argument, and returns the standard uncertainty of the mean of the dataset.
    '''
    N = len(data)
    return standard_deviation(data) / np.sqrt(N)
def t_prime(A, dA, B, dB):
    '''
    Returns the value of t prime. Requires 4 arguments in this order: measurement A, uncertainty of measurement A, measurement B, uncertainty of measurement B.
    '''
    return abs(A - B) / np.sqrt(dA**2 + dB**2)






### Uncertainty propagation

In [ ]:
def dsum(x, dx, y, dy):
    return np.sqrt(dx**2+dy**2)

def dsub(x, dx, y, dy):
    return np.sqrt(dx**2+dy**2)

def dmult(x, dx, y, dy):
    return x*y*np.sqrt((dx/x)**2+(dy/y)**2)

def ddiv(x, dx, y, dy):
    return x/y*np.sqrt((dx/x)**2+(dy/y)**2)

def dpwr(x, dx, n):
    return n*dx*(x**n)/x

def dln(x, dx):
    return dx/x